In [1]:
import pandas as pd

file_path = 'TimeEdit_GIG1003_ASAS_PEMBUDAYAAN_KEUSAHAWANAN_6_Module_7_2024-09-20_21_48.csv'
   
# Load the CSV and skip the first 3 lines, using the fourth line as the header
df = pd.read_csv(file_path, skiprows=3)

# Trim all column names
df.columns = df.columns.str.strip()

# Create 'Day' column from 'Begin date'
df['Begin date'] = pd.to_datetime(df['Begin date'], format='%d/%m/%Y')
df['Day'] = df['Begin date'].dt.strftime('%a')

# Clean data: Drop unnecessary columns and duplicates
df_cleaned = df.drop(columns=['Begin date', 'End date'], errors='ignore').drop_duplicates()

# Convert 'Begin time' and 'End time' to formatted time and calculate 'Duration'
df_cleaned['Begin time'] = pd.to_datetime(df_cleaned['Begin time'], format='%H:%M').dt.strftime('%I:%M %p')
df_cleaned['End time'] = pd.to_datetime(df_cleaned['End time'], format='%H:%M').dt.strftime('%I:%M %p')

# Correct duration calculation
df_cleaned['Duration'] = (pd.to_datetime(df_cleaned['End time'], format='%I:%M %p') - 
                          pd.to_datetime(df_cleaned['Begin time'], format='%I:%M %p')).dt.total_seconds() / 60

df_cleaned['Duration'] = df_cleaned['Duration'].apply(lambda x: f'{int(x // 60):02}:{int(x % 60):02}')

# Filter out rows where 'Activity Type (exam)' is not null
df_cleaned_filtered = df_cleaned[df_cleaned['Activity Type (exam)'].isnull()]

# Split 'Module Offering' by comma and dash to extract Course Code and Occurrence
df_cleaned_filtered = df_cleaned_filtered.assign(
    **{'Module Offering': df_cleaned_filtered['Module Offering'].str.split(',')}
).explode('Module Offering').reset_index(drop=True)

df_cleaned_filtered[['Course Code', 'Year', 'Semester', 'Occurrence']] = df_cleaned_filtered['Module Offering'].str.split('/', expand=True)

# Drop unneeded columns and rename 'Module' to 'Course Name'
df_cleaned_filtered = df_cleaned_filtered.drop(columns=['Module Offering', 'Year', 'Semester', 'Activity Type (exam)'])
df_cleaned_filtered = df_cleaned_filtered.rename(columns={'Module': 'Course Name'})

# Group by Course Code, Occurrence, and Activity
grouped_data = df_cleaned_filtered.groupby(['Course Code', 'Occurrence']).apply(
    lambda x: x[['Activity', 'Day', 'Begin time', 'End time', 'Duration', 'Room']].to_dict(orient='records')
).reset_index().rename(columns={0: 'Activities'})

# Create the final JSON structure
result_json = grouped_data.groupby('Course Code').apply(
    lambda x: {
        'Course Name': df_cleaned_filtered[df_cleaned_filtered['Course Code'] == x['Course Code'].iloc[0]]['Course Name'].iloc[0],
        'Course Code': x['Course Code'].iloc[0],
        'Occurrences': [{'Occurrence': occ, 'Activities': acts} for occ, acts in zip(x['Occurrence'], x['Activities'])]
    }
).reset_index(drop=True).to_json(orient='records', indent=4)

# Save the result to a JSON file
output_path = 'course_data.json'
with open(output_path, 'w') as file:
    file.write(result_json)

print(f"JSON saved to {output_path}")


C:\Users\User\anaconda3\Lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


JSON saved to course_data_final_fixed.json


C:\Users\User\AppData\Local\Temp\ipykernel_6992\3982863672.py:43: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  grouped_data = df_cleaned_filtered.groupby(['Course Code', 'Occurrence']).apply(
C:\Users\User\AppData\Local\Temp\ipykernel_6992\3982863672.py:48: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  result_json = grouped_data.groupby('Course Code').apply(
